In [1]:
setwd(paste0(Sys.getenv('R_SOURCES'), '/house_prices'))
getwd()

[1] "/mnt/hdd/develop/kaggle/R_sources/house_prices"

In [13]:
source('main.R')

house_prices$helpers$import_libs()

combined_dataset <- house_prices$helpers$load_data()

numeric_data <- 
    combined_dataset %>% 
    select(house_prices$helpers$get_numeric_colnames(combined_dataset), -Id) %>%
    select(-one_of(house_prices$attributes_selection$discared_num_vars))

numeric_data %>% colnames %>% sort

[1] "BedroomAbvGr" "BsmtFinSF1"   "BsmtFullBath" "BsmtUnfSF"    "Fireplaces"  
 [6] "FullBath"     "GarageArea"   "GarageCars"   "GarageYrBlt"  "GrLivArea"   
[11] "HalfBath"     "LotArea"      "LotFrontage"  "MasVnrArea"   "MoSold"      
[16] "OpenPorchSF"  "OverallCond"  "OverallQual"  "SalePrice"    "TotalBsmtSF" 
[21] "TotRmsAbvGrd" "WoodDeckSF"   "X1stFlrSF"    "X2ndFlrSF"    "YearBuilt"   
[26] "YearRemodAdd" "YrSold"

In [68]:
best_transformation <-
    numeric_data %>%
    gather(var, x) %>%
    filter(!is.na(x)) %>%
    mutate(
        log = log(x + 1),
        sqrt = sqrt(x)
    ) %>%
    gather(predictor, value, -var) %>%
    group_by(var, predictor) %>%
    mutate(
         value_normed = (value - mean(value)) / sd(value)
    ) %>%
    group_by(var, predictor, value_normed) %>%
    arrange(var, predictor, value_normed) %>%
    summarise(
        k = n()
    ) %>%
    mutate(
        empirical = cumsum(k) / sum(k),
        theoretical = pnorm(value_normed),
        diff_L2 = k*(empirical - theoretical)**2
    ) %>%
    group_by(var, predictor) %>%
    summarise(
        L2_distance = sum(diff_L2)
    ) %>%
    group_by(var) %>%
    nest %>%
    mutate(
        best_predictor = map(data, function(df) {
            best <- df %>% arrange(L2_distance) %>% head(1)
            x <- df %>% filter(predictor == 'x')
            score <- 100 * (x$L2_distance - best$L2_distance) / x$L2_distance
            data_frame(predictor = best$predictor, score = score)
        })
    ) %>%
    select(var, best_predictor) %>%
    unnest(best_predictor) %>%
    filter(predictor != 'x' & score > 5)

In [69]:
best_transformation

var,predictor,score
BedroomAbvGr,log,22.36315
BsmtUnfSF,sqrt,80.12889
GarageCars,log,24.95101
GrLivArea,log,92.16162
LotArea,log,73.81288
LotFrontage,sqrt,44.25830
MoSold,log,70.75453
OverallCond,log,16.23102
OverallQual,log,29.98095
SalePrice,log,91.29422


In [70]:
best_transformation$var

[1] "BedroomAbvGr" "BsmtUnfSF"    "GarageCars"   "GrLivArea"    "LotArea"     
 [6] "LotFrontage"  "MoSold"       "OverallCond"  "OverallQual"  "SalePrice"   
[11] "TotRmsAbvGrd" "X1stFlrSF"

In [72]:
numeric_data %>%
    select(one_of(best_transformation$var)) %>%
    gather(var, value)

var,value
BedroomAbvGr,3
BedroomAbvGr,3
BedroomAbvGr,3
BedroomAbvGr,3
BedroomAbvGr,4
BedroomAbvGr,1
BedroomAbvGr,3
BedroomAbvGr,3
BedroomAbvGr,2
BedroomAbvGr,2
